# Random Forest

Una de las últimas tendencias dentro de lo que serían los modelos de inteligencia artificial viene a resumirse como "el conocimiento del conjunto o la multitud". Lo que viene a definir esta frase, un tanto popular, es el uso de multitud de modelos denominados "débiles" en un metaclasificador. El objetivo es generar un modelo "fuerte" en base al conocimiento extraído por los modelos "débiles". Por ejemplo, aunque se detallará más adelante, en un *Random Forest* se desarrollan múltiples *Decision Trees* mucho más simples. La combinación de estos en el *Random Forest* excede el rendimiento de cualquiera de los modelos individuales. Los modelos surgidos de está manera, como metaclasificadores o metaregresores  reciben el nombre genérico de modelos *Ensemble*.

Un hecho a destacar es el hecho de que estos modelos pueden no limitarse unicamente a los arboles de decisión, por contra pueden componerse de cualquier tipo de modelo de aprendizaje automático que se ha visto previamente. Incluso pueden ser modelos mixtos donde no todos los modelos se hayan obtenido de la misma manera, si no que pueden ser creados mediante el uso combinado de varias técnicas como pueden ser el K-NN, SVM, etc.

En la presente unidad se va a explorar se verán dos de las técnicas más habituales dentro de estos modelos *ensemble* como son el _Random Forest_, y _XGBoost. El primero se verá en base a ejemplos, mientras que el segundo se dejará sólo apuntado por ser una de las mayores tendencias actualmente.Otras alternativas se abordarán en capítulos posteriores.


### Random Forest
Este algoritmo, propuesto por Breitman y Cutler en 2006 a instancias de una publicación anterior de Ho de 1995 (_Random Subspaces_), es el paradigma de técnica de ensemble. El algoritmo une en un ensemble un conjunto de clasificadores sencillos que toman la forma de *Decision Trees*. Estos clasificadores son entrenados siguiendo una aproximación de *bagging*, y por lo tanto se pueden entrenar cada uno de forma paralela. Para combinar la salidad de los algoritmos se hace para los problemas de clasificación mendiante la opción más votada entre los "expertos" o, si es un problema de regresión, mediante la media aritmética de las respuestas. 

Es un algoritmo que necesita el ajuste de muy pocos hiperparámetros para obtener muy buenos resultados en casi cualquier tipo de problema. En general, el valor más importante es el número de estimadores y por tanto el número de particiones que se va a hacer del conjunto de entrenamiento. Varios autores apuntan que ese número de estimadores debiera de ser *$\sqrt{\textrm{número características}}$* para problemas de clasicación, y *$\frac{\textrm{número características}}{3}$ para problemas de regresión. Aun así, también apunta que la técnica saturaría entre 500 y 1000 árboles y por mucho que se aumente no mejoraría resultados. Si bien este último dato solo ha sido probado de manera empírica en determinados conjuntos de datos y por lo tanto debe de ser tomado con cuidado al no tener una justificación matemática.

A mayores del proceso habitual de *bagging*, los *Random Forest* también incluyen un segundo mecanismo de división. Una vez seleccionados los patrones que formaran parte del conjunto de entrenamiento del árbol de decisión, solamente un subconjunto de características (*features*) aleatorias están disponibles para cada nodo del árbol. Esto hace crecer la diversidad de los árboles del bosque y lo que consigue es centrarse en el rendimiento global con una pequeña varianza en los resultados. Este mecanísmo permite evaluar cuantitativamente el rendimiento individual de cada árbol que forma parte del bosque y sus variables. Por tanto, se puede medir la importancia de cada variable. Está medida que calibra la participación de cada variable en nodos del árbol en la toma de decisiones se denomina impureza y viene a medir la diferencia entre las diferentes ramas del árbol cuando se hace la partición de los ejemplos. En ocasiones, esta misma medida es utilizada a modo de medida para la selección de variables tomando la medida en todos los árboles del bosque de la participación e importancia mediante un filtrado como los vistos en la unidad anterior.

Para el calculo de esa medida de impureza, existen diferentes aproximaciones. Por ejemplo, `scikit learn` utiliza una medida que denomina **Gini**. Esta última es la probabilidad de clasificar incorrectamente un elemento elegido al azar en el conjunto de datos si se etiquetara aleatoriamente según la distribución de clases en el conjunto de datos. Se calcula como:
$$G = \sum_{i=1}^C p(i) * (1 - p(i))$$

siendo $C$ el número de clases y $p(i)$ la probabilidad de seleccionar al azar un elemento de la clase $i$. Se puede ver un buen ejemplo de como calcular la impureza de las ramas en el siguiente [enlace](https://victorzhou.com/blog/gini-impurity/)

A continuación, sobre el ejemplo que se viene utilizando en esta unidad, se ejecutará un modelo de *Random Forest* con la implementación de `scikit learn`. Destacar como parámetros más importante de dicha implementación:

- ***n_estimator***, que marca el número de árboles que se van a generar o el número de particiones de *bagging*.
- ***criterion***, medida de impureza de los nodos. Por defecto se usa Gini aunque puede cambiarse por la entropía ganada.
- ***max_depth***, permite limitar la profundidad máxima de los árboles para así limitar el número de variables a usar.
- ***min_sample_split***, para cada árbol de decisión, cuantos patrones son necesario para realizar una división interna en los *Decision Trees*.
- ***bootstrap***, puede utilizar la aproximación de *bagging* o *bootstrap* para construir los árboles pero si esta propiedad es falsa, entonces usa todo el conjunto de entrenamiento para generar los árboles. En caso de tener un valor True, se tienen en cuenta las siguientes propiedades:
    + ***max_samples***, número de ejemplos a extraer del conjunto original para construir el conjunto de entrenamiento del estimador, el valor por defecto es igual al número de patrones pero recuerde que puede extraerse varias veces el mismo ya que es una selección con remplazo dando variabilidad.
    + ***oob_score***, medida de *out of bag* para estimar la generalización. Aquellas muestras que no han formado parte del entrenamiento de un estimador se pueden usar para calcular una medida de validación y, promediarla entre todos los estimadores para saber como de general es el bosque construído.
 

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier

breast_cancer = load_breast_cancer()

modelo_RF = RandomForestClassifier(n_estimators=500,n_jobs=-1)

modelo_RF.fit(breast_cancer["data"],breast_cancer["target"])

A continuación, una vez entrenado el modelo, se puede comporobar el nivel de impureza obtenido para cada una de las frecuencias que se ha calculado con el el algoritmo Gini, como una media de las obtenidas entre los arboles que componen el bosque

In [ ]:
for name, score in zip(breast_cancer["feature_names"],modelo_RF.feature_importances_):
    print(name,score)

In [ ]:
import pandas as pd

res=pd.DataFrame(sorted(zip(map(lambda x: round(x, 4), modelo_RF.feature_importances_), breast_cancer.feature_names), 
             reverse=True),)
res.columns = ['Importance', 'Feature']

print(res)

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

plt.rcParams["figure.figsize"]=10,10
res.plot.barh(x='Feature')
plt.xlabel("Gini Gain")
plt.ylabel("Característica")
plt.title("Importancia de las características ")

apuntar que , como se ve en el gráfico, este valor determina que la mayor parte de la información se concentra en algunas de las frecuencias utilizadas. Es por ello, que como se comento antes, se podría realizar un filtrado de la información como el visto en la unidad anterior en base a este valor. 

### XGBoost (eXtreme Gradient Boosting)

Finalmente, en este último apartado destacar nuevamente el Gradient Boosting, en concreto, una implementación que en los últimos años se ha hecho muy famosa por su versatilidad y rapidez. Esta implementación que se conoce como ***XGBoost (eXtreme Gradient Boosting)*** , que ha destacado sobre todo en competiciones como en la plataforma Kaggle por su rápidez en la obtención de resultados y robustez de los mismos. 

El ***XGBoost*** será un ensemble similar al de los Random Forest pero utiliza un clasificador base diferente conocido como CART (classification and regression trees) en lugar de *Decision Trees*. Este cambio viene de la mano de la necesidad del algoritmo de obtener la  probabilidad de las decisiones, al igual que ocurría con el *Gradient Tree Boosting*. El otro de los cambios fundamentales de este algotimo es, ya que está basado en el * Gradiente Tree Boosting*, es el cambio de la estrategia de *bagging* por la de *boosting* para la creación de los conjuntos de entrenamiento de los clasificadores.

Posteriormente, está técnica realiza una aproximación de entrenamiento aditivo cuyos pesos se van ajustanto en base a un **Gradiente Descendente** sobre una función de *loss* a definir. Sumando la función de *loss* con el término de regularización, se puede calcular hasta la segunda derivada de las funciones con el fin de actualizar los pesos de la clasificación realizado por los diferentes árboles. El cálculo de este gradiente, permite por lo tanto el ajuste de los valores de los clasificadores que se generan a continuación de uno dado con el fin de que los pesos permitan focalizar la atención en los patrones que incorrectamente clasificados. Los detalles matemáticos de la implementación se pueden consultar en este [enlace](https://xgboost.readthedocs.io/en/stable/tutorials/model.html).

Al diferencia del resto de aproximaciones que hemos visto, el `xgboost` no se encuentra actualmente implementado en `scikit learn`. POr este motivo, se deberá de instalar la versión de referencia si no está ya presente en la máquina.

In [ ]:
try:
    import xgboost as xgb
except ModuleNotFoundError:
    !pip install xgboost
    import xgboost as xgb

Tras esa instalación, se podría hacer uso de la librería como se ve en el ejemplo siguiente. En primer lugar, para hace uso de está librería es necesario hacer una adapción de los datos de entrada al formato [LIBSVM](https://xgboost.readthedocs.io/en/stable/tutorials/input_format.html). Existen varias formas de cargar los datos desde numpy, scipy o pandas, para mayores detalles sobre este punto y para aplicarlo a diferentes problemas, se pude consultar el siguiente enlace [enlace](https://xgboost.readthedocs.io/en/stable/python/python_intro.html). En este caso concreto, el ejemplo está almacenado en un array de `numpy` con lo que para la transformación de los datos bastaría con:

In [ ]:
# preparar las matrices para usarlas con el formato LIBSVM
from sklearn.metrics import train_test_split

train_input, test_input, train_output, test_output = train_test_split(breast_cancer["data"], breast_cancer["target"],test_size=0.2)

dtrain = xgb.DMatrix(train_input, label=train_outputs)
dtest = xgb.DMatrix(test_inputs, label=test_outputs)

Una vez realizada está adaptación de los datos, se puede proceder con el entrenamiento de un modelo de la librería `xgboost`. Para ello sólo hará falta llamar a la función train con los parámetros correspondientes. Dentro de estos parámetros destacan:

- **eta**, término que determinará la compresión de los pesos tras cada nueva etapa del *boosting*. Toma valores entre 0 y 1.
- **max_depth**, profundidad máxima de los árboles tiene por defecto un valor de 6 incrementarlo lo que hará será permitir modelos más complejos
- **gamma**, parámetro que controla la reducción mínima de pérdidas necesaria para realizar una nueva partición en un nodo hoja del árbol. Cuanto mayor sea será más conservador
- **alpha** y **lambda**, son los parámetros que controlan la regulación L1 y L2 respectivamente.
- **objective**, establece la fución de loss a ser utilizada que puede ser una de las predefinidas, las cuáles se pueden consultar en este [enlace](https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster)

A mayores solo es necesario establecer el número máximo de iteraciones del proceso de boosting como se ve en el siguiente ejemplo con 40 rondas.

In [ ]:
# Especificar los parámetros del modelo
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 40

# entrenar el modelo correspondiente
xgb_model = xgb.train(param, dtrain, num_round)

***NOTA***

En caso de usarse un conjunto de validación, este debe de pasarse en el parámetro *evals* de la función de entrenamiento. A mayores, y sólo cuando el mencionado parámetro *evals* está definido, se puede establecer las rondas para la parada previa con el parámetro *early_stopping_rounds* de la función de entrenamiento. El código sería similar a:
``` python
    evals = xgb.DMatrix(val_inputs, label=val_outputs)
    xgb_model = xgb.train(param, dtrain, num_round, evals=evals, early_stopping_rounds=10)
```

El valor proporcionado en la salida se corresponde con la suma de las salidas de los árboles, estándo está entre 0 y 1 apra la pertenencia a una determinada clase. Como se trata de un es una clase binaria, simplemente se establece un límite de 0.5 a la salida para determinar cual es la respuesta.

In [ ]:
from sklearn.metrics import accuracy_score

print(f"{xgb_model.predict(dtest)}")

# Evaluar la salida
acc = accuracy_score(xgb_model.predict(dtest)>0.5,test_outputs)
print(f"XGB: {(acc*100):.4f}%")

Finalmente, al igual que en el caso de los *Random Forest* es posible identidicar la importancia y pintarla para cada una de las variables en la clasificación. Con el siguiente código se pude ver dicho marcador ordenado decendentemente

In [ ]:
# imprimir la importancia de las características
xgb.plot_importance(xgb_model)